In [1]:
!pip install -q transformers langchain huggingface_hub accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.5/238.5 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.5 MB/s eta 0:00:00


In [2]:
from huggingface_hub import login
login()

In [3]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

In [4]:
model = "meta-llama/Llama-2-7b-chat-hf"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [6]:
pipeline = transformers.pipeline("text-generation", model=model, tokenizer= tokenizer, torch_dtype=torch.bfloat16, device_map="auto", max_new_tokens = 512, do_sample=True, top_k=10, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id, )

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [7]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [8]:
from langchain import PromptTemplate, LLMChain
template = """ Create a SQL query snippet using the below text: ```{text}``` Just SQL query: """
prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
text = """ Extract all the unique values from column "age" """

In [9]:
print(llm_chain.run(text))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 ``` SELECT DISTINCT age FROM table_name ```  Explanation:  The `SELECT DISTINCT` clause is used to retrieve only distinct values from a column. It eliminates duplicate values and returns only unique values. In this case, we are using it on the "age" column of the table `table_name`.


In [10]:
print(llm_chain.run("Get the complete record(Customer name, product name) from both tables([CustomerDetail],[ProductDetail]), if no match is found in any table then show NULL"))


 ```SELECT CustomerName, ProductName FROM CustomerDetail  LEFT JOIN ProductDetail ON CustomerName = ProductName WHERE CustomerName IS NOT NULL AND ProductName IS NOT NULL```  Explanation:  In this query, we are joining two tables `CustomerDetail` and `ProductDetail` on the common column `CustomerName`. If any record is found in either table, the join will return the complete record. If no match is found in any table, the query will return `NULL`. 

The query uses the `LEFT JOIN` operator, which returns all records from the left table (`CustomerDetail`) and matches them with the right table (`ProductDetail`). If there is no match in the right table, the resulting record will have a `NULL` value in the `ProductName` column.

Note that we also use the `IS NOT NULL` condition in the `WHERE` clause to ensure that the query only returns records where the `CustomerName` and `ProductName` columns are not `NULL`. This helps to avoid returning incomplete records or `NULL` values in the result se